In [12]:
from datasets import load_dataset

dataset = load_dataset("hynky/czech_news_dataset", split="train")

Found cached dataset parquet (/home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-2bef7073f90328ac/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [8]:
# get the indices of the top 10 features
import pickle
import sys
sys.path.append("../Modelling/Baseline/")
models = {
    "50k": {
        "category": "rgz92chu",
        "day_of_week": "xyg91uvw",
        "server": "8yvpnynw",
        "authors_cum_gender": "evkcoi0s"
    },
    "200k": {
        "category": "fbf1hiz7",
        "day_of_week": "er8hxbbd",
        "server": "irpb5y6l",
        "authors_cum_gender": "evkcoi0s"
    }
}

data = {
    "50k": pickle.load(open("../Modelling/Baseline/tfidf/small/vectorizer.pkl", "rb")),
    "200k": pickle.load(open("../Modelling/Baseline/tfidf/base/vectorizer.pkl", "rb"))
}






In [13]:
import numpy as np
top_table = {}
k = 4
for size, task in models.items():
    top_table[size] = {}
    vectorizer = data[size]
    for feature, model_id in task.items():
        if model_id is None:
            top_table[size][feature] = None
            continue
        model = pickle.load(open(f"../Modelling/Baseline/models/{feature}/{model_id}/model.pkl", "rb"))
        per_cls = {}
        for cls_i, cls in enumerate(dataset.features[feature].names[1:]):
            top_k = np.argsort(model.coef_[cls_i])[-k:]
            # -5 because we have there non tfidf features
            top_k_vals = [(vectorizer.get_feature_names_out()[i-5], model.coef_[cls_i][i]) for i in top_k]
            per_cls[cls] = top_k_vals
        top_table[size][feature] = per_cls

In [14]:
table = top_table["200k"]

In [28]:
# normalize following unicode character
from unicodedata import normalize
from html import unescape
uni = '〜'
uni_n = normalize('NFKC', unescape(uni))


In [34]:
# normalize uni to character that can be used in latex
uni_n = uni_n.replace("~", "\\textasciitilde")

In [35]:
uni_n

'〜'

In [36]:
cat_sort_order = [
    "Home", 
    "Sport",
    "World",
    "Culture",
    "Economics",
    "Crime",
    "Technology",
    "Tabloid News and Curiosities",
    "Lifestyle",
    "Car",
    "Science",
    'Comments',
    'Travel',
    'Finance',
    'Entrepreneurship',
    'Housing',
    'Coronavirus',
    'Business',
    'Interviews',
    'Podcasts',
    'Tabloid News and Curiosities',
    'Literature',
    'Christmas',
    'Fine Arts',
    'Bicycle'
]

In [37]:
translate_task = {
    "category": "Category",
    "day_of_week": "Day of week",
    "server": "Server",
    "authors_cum_gender": "Gender"
}

In [44]:
dst = load_dataset("hynky/czech_news_dataset")

Found cached dataset parquet (/home/kydliceh/.cache/huggingface/datasets/hynky___parquet/hynky--czech_news_dataset-62438212bae7f23d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/6 [00:00<?, ?it/s]

'" '

In [48]:
dst["test"].features["category"] = dst["test"].features["category"].names

In [46]:
from ..dataset_creation.postprocessing_utils import normalize_text
dst = dst["train"].map(lambda x: {"content_norm": normalize_text(x["content"])})

ImportError: attempted relative import with no known parent package

In [45]:
import pandas as pd

for task in table:
    tbl = table[task]
    # remove values and keep only the features
    new_task = {}
    for cls, features in tbl.items():
        sorted_features = sorted(features, key=lambda x: x[1], reverse=True)
        new_task[cls] = [x[0] for x in sorted_features]

    df = pd.DataFrame(new_task)
    if task == "category":
        df = df[cat_sort_order]
    name = translate_task[task]
    # print latex but start from 1
    transposed = df.T
    transposed = transposed.rename(columns={i: i+1 for i in range(k)})
    # unescape html
    transposed = transposed.applymap(lambda x: unescape(x))

    # write to file
    with open(f"top{k}_{task}.tex", "w") as f:
        latex = transposed.to_latex(caption=f"Top {k} features for {name}", label=f"tab:top{k}_{task}", column_format="l" + "r" * k)
        f.write(latex)
        print(latex)


\begin{table}
\centering
\caption{Top 4 features for Category}
\label{tab:top4_category}
\begin{tabular}{lrrrr}
\toprule
{} &               1 &                2 &              3 &               4 \\
\midrule
Home                         &         mf dnes &     radiožurnálu &       idnes.cz &           klaus \\
Sport                        &         fotbalu &              nhl &        turnaji &           kluby \\
World                        &             osn &          . úřady &      : reuters &   vyšetřovatelé \\
Culture                      &           album &               čt &       koncertu &          snímek \\
Economics                    &           banka &         analytik &            řsd &       ekonomiky \\
Crime                        &  mluvčí policie &       řekl právu &  miliónů korun &     řekla právu \\
Technology                   &           herní &               pc &        windows &           . hra \\
Tabloid News and Curiosities &            miss &         novinká

/COMP.TMP/kydliceh_8485/ipykernel_1976287/3034776479.py:23: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = transposed.to_latex(caption=f"Top {k} features for {name}", label=f"tab:top{k}_{task}", column_format="l" + "r" * k)
/COMP.TMP/kydliceh_8485/ipykernel_1976287/3034776479.py:23: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = transposed.to_latex(caption=f"Top {k} features for {name}", label=f"tab:top{k}_{task}", column_format="l" + "r" * k)
/COMP.TMP/kydlic